In [ ]:
#데이터 정리
import pandas as pd
from astropy.table import Table
import numpy as np
import urllib.request
from astropy.io import fits

# 1. 파일 URL 및 이름 정의
fits_url = 'https://data.sdss.org/sas/dr17/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits'
local_filename = 'dapall-correct.fits'

# 2. 파일 다운로드
print(f"'{local_filename}' 파일을 다운로드합니다...")
urllib.request.urlretrieve(fits_url, local_filename)
print("다운로드 완료.")

# 3. fits.open()으로 파일을 직접 열기
with fits.open(local_filename) as hdul:
    # 4. 2번 HDU의 데이터(hdul[2].data)를 직접 Table 객체로 변환
    print("\n파일의 2번 HDU에서 데이터를 직접 추출합니다...")
    dat = Table(hdul[2].data)

# 5. 다차원 배열 컬럼 필터링 (오류 방지)
names_1d = [name for name in dat.colnames if len(dat[name].shape) <= 1]
filtered_dat = dat[names_1d]

# 6. 필터링된 데이터를 DataFrame으로 변환
manga_df = filtered_dat.to_pandas()

# --- 💡💡💡 수정된 부분 💡💡💡 ---

# 7. 로드된 모든 데이터 확인하기
#    pandas가 모든 열을 보여주도록 옵션 설정
pd.set_option('display.max_columns', None)

print(f"\n✅ SUCCESS: 총 {len(manga_df.columns)}개의 컬럼을 가진 {len(manga_df)}개의 은하 데이터를 로드했습니다.")
print("데이터 샘플 (전체 컬럼):")
print(manga_df.head())

'dapall-correct.fits' 파일을 다운로드합니다...
다운로드 완료.

파일의 2번 HDU에서 데이터를 직접 추출합니다...

✅ SUCCESS: 총 68개의 컬럼을 가진 10782개의 은하 데이터를 로드했습니다.
데이터 샘플 (전체 컬럼):
   PLATE  IFUDESIGN    PLATEIFU   MANGAID  DRPALLINDX  MODE  \
0   7443       1902   7443-1902  12-49536        4323  CUBE   
1   7443      12704  7443-12704  12-84731        4320  CUBE   
2   7443       1901   7443-1901  12-84620        4322  CUBE   
3   7443       3702   7443-3702  12-84670        4325  CUBE   
4   7443       3704   7443-3704  12-84617        4327  CUBE   

                   DAPTYPE  DAPDONE       OBJRA     OBJDEC       IFURA  \
0  VOR10-MILESHC-MASTARSSP     True  231.991104  42.971207  231.991104   
1  VOR10-MILESHC-MASTARSSP     True  232.461060  42.628967  232.461060   
2  VOR10-MILESHC-MASTARSSP     True  231.042603  42.068722  231.042603   
3  VOR10-MILESHC-MASTARSSP     True  230.598343  43.367775  230.598343   
4  VOR10-MILESHC-MASTARSSP     True  231.478760  41.909771  231.478760   

      IFUDEC  MNGTARG1  MNGTARG2 

In [ ]:
# 데이터 조정 및 학습
# ## 0단계: 라이브러리 불러오기 및 설치
# ######################################################################
print(">>> 0단계: 라이브러리 불러오기 및 필수 라이브러리 설치...")

# 필수 라이브러리가 설치되어 있는지 확인하고, 없으면 설치합니다.
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
    except ImportError:
        print(f"'{package}' 라이브러리가 없어 설치합니다...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 필요한 라이브러리 목록
required_packages = ['pandas', 'numpy', 'astropy', 'scikit-learn', 'imbalanced-learn']
for pkg in required_packages:
    install_package(pkg)

import pandas as pd
import numpy as np
import urllib.request
from astropy.table import Table
from astropy.io import fits
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from imblearn.over_sampling import SMOTE


print("\n라이브러리 로드 및 확인 완료.\n")


# ## 1단계: dapall 파일 다운로드 및 데이터 통합
# ######################################################################
print(">>> 1단계: dapall.fits 파일을 다운로드하고 데이터를 통합합니다...")
local_filename = 'dapall-v3_1_1-3.1.0.fits'
try:
    with open(local_filename, 'rb') as f:
        print(f" - '{local_filename}' 파일이 이미 존재합니다.")
except FileNotFoundError:
    print(f" - '{local_filename}' 다운로드 중...")
    fits_url = 'https://data.sdss.org/sas/dr17/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits'
    urllib.request.urlretrieve(fits_url, local_filename)

try:
    with fits.open(local_filename) as hdul:
        # --- 1번 HDU (모양/구조 정보) 로드 ---
        dat1 = Table(hdul[1].data)
        names1 = [name for name in dat1.colnames if len(dat1[name].shape) <= 1]
        df_hdu1 = dat1[names1].to_pandas()
        df_hdu1 = df_hdu1[['MANGAID', 'OBJRA', 'OBJDEC', 'NSA_SERSIC_N', 'NSA_ELPETRO_BA', 'NSA_SERSIC_TH50']]

        # --- 2번 HDU (운동/물리 정보) 로드 ---
        dat2 = Table(hdul[2].data)
        names2 = [name for name in dat2.colnames if len(dat2[name].shape) <= 1]
        df_hdu2 = dat2[names2].to_pandas()
        df_hdu2 = df_hdu2[['MANGAID', 'Z', 'STELLAR_SIGMA_1RE', 'SFR_TOT', 'SB_1RE']]

        # --- MANGAID 문자열 정리 ---
        for df_ in [df_hdu1, df_hdu2]:
            if df_['MANGAID'].dtype == 'object' and isinstance(df_['MANGAID'].iloc[0], bytes):
                df_['MANGAID'] = df_['MANGAID'].str.decode('utf-8').str.strip()

        # --- 두 데이터프레임 병합 ---
        dapall_df = pd.merge(df_hdu1, df_hdu2, on='MANGAID')

    print("데이터 로드 및 통합 성공.\n")

except Exception as e:
    print(f"🚨 파일 처리 중 오류: {e}")
    # 프로그램 중단
    sys.exit()


# ## 2단계: 피처 및 라벨 준비
# ###############################################################
print(">>> 2단계: 피처 및 라벨 준비...")
features = ['NSA_SERSIC_N', 'NSA_ELPETRO_BA', 'STELLAR_SIGMA_1RE', 'SFR_TOT', 'Z', 'SB_1RE']
type_target = 'galaxy_type'
size_target = 'NSA_SERSIC_TH50'

# 결측치 제거
df_full = dapall_df[features + [size_target]].dropna().copy()

# 3개 클래스 (타원, 나선, 불규칙)로 분류하는 함수
def classify_galaxy_3types(row):
    n = row['NSA_SERSIC_N']
    sigma = row['STELLAR_SIGMA_1RE']
    sfr = row['SFR_TOT']

    if n > 2.5 and sigma > 120 and sfr < 0.01:
        return 'Early-type (Elliptical)'
    elif n < 2.5 and sfr > 0.01:
        return 'Late-type (Spiral)'
    else:
        return 'Irregular'

df_full[type_target] = df_full.apply(classify_galaxy_3types, axis=1)

# 학습용 데이터 최종 정리
df_train = df_full.copy()
print("피처 및 라벨 준비 완료.\n")


# ## 3단계: 2개의 모델 학습 (최소 클래스 3배 증강)
# ###############################################################
print(">>> 3단계: 2개의 모델을 학습합니다...")

# --- 모델 1: 모양 예측 (분류 모델) ---
print("  - 모델 1 (모양 예측) 학습 중...")
X_type = df_train[features]
y_type = df_train[type_target]

# 데이터를 훈련셋과 테스트셋으로 분리 (stratify로 클래스 비율 유지)
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_type, y_type, test_size=0.2, random_state=42, stratify=y_type)

print(f"  - 오버샘플링 전 훈련 데이터 클래스 분포:\n{y_train_t.value_counts()}")

# 가장 적은 클래스만 3배 증강하는 로직
# 1. 기존 클래스별 샘플 수를 딕셔너리로 저장
strategy = y_train_t.value_counts().to_dict()

# 2. 가장 샘플 수가 적은 클래스(minority class)의 이름과 개수 찾기
minority_class_name = y_train_t.value_counts().idxmin()
minority_class_count = strategy[minority_class_name]

# 3. 최소 클래스의 목표 샘플 수를 3배로 설정 (나머지는 그대로)
strategy[minority_class_name] = minority_class_count * 3
print(f"\n  - '{minority_class_name}' 클래스를 {minority_class_count}개 -> {strategy[minority_class_name]}개로 증강합니다.")

# 4. 수정된 전략으로 SMOTE 적용
smote = SMOTE(sampling_strategy=strategy, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_t, y_train_t)
print(f"\n  - 오버샘플링 후 훈련 데이터 클래스 분포:\n{y_train_resampled.value_counts()}")

# 오버샘플링된 데이터로 모델 학습
model_type = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model_type.fit(X_train_resampled, y_train_resampled)

# 테스트 데이터로 모델 평가
y_pred_t = model_type.predict(X_test_t)
print(f"\n  - 모델 1 학습 완료. 테스트 정확도: {accuracy_score(y_test_t, y_pred_t):.4f}")
print("  - 상세 분류 리포트:\n", classification_report(y_test_t, y_pred_t))

# --- 모델 2: 크기 예측 (회귀 모델) ---
print("  - 모델 2 (크기 예측) 학습 중...")
X_size = df_train[features]
y_size = df_train[size_target]
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_size, y_size, test_size=0.2, random_state=42)
model_size = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_size.fit(X_train_s, y_train_s)
mse = mean_squared_error(y_test_s, model_size.predict(X_test_s))
print(f"  - 모델 2 학습 완료. 평균 제곱 오차(MSE): {mse:.4f}\n")


# ## 4단계: 사용자 입력 및 통합 예측
# ###############################################################
print(">>> 4단계: 사용자 입력 및 통합 예측 파트입니다.")

def predict_galaxy_all(sersic_n, ba_ratio, sigma, sfr, redshift, sb_1re):
    """사용자가 입력한 6개 값으로 은하 모양과 크기를 모두 예측"""
    input_data = pd.DataFrame({
        'NSA_SERSIC_N': [sersic_n], 'NSA_ELPETRO_BA': [ba_ratio],
        'STELLAR_SIGMA_1RE': [sigma], 'SFR_TOT': [sfr],
        'Z': [redshift], 'SB_1RE': [sb_1re]
    })

    # 모델 1로 모양 예측
    type_prediction = model_type.predict(input_data)[0]
    type_probabilities = model_type.predict_proba(input_data)[0]

    # 모델 2로 크기 예측
    size_prediction = model_size.predict(input_data)[0]

    print("\n--- 통합 예측 결과 ---")
    print(f"➡️ 은하 모양: '{type_prediction}'일 가능성이 높습니다.")
    print(f"➡️ 은하 크기: 유효반경 약 {size_prediction:.2f} arcsec로 예측됩니다.")

    print("\n[상세] 모양별 확률:")
    for i, class_name in enumerate(model_type.classes_):
        print(f"- {class_name}: {type_probabilities[i]*100:.2f}%")

    return type_prediction, size_prediction

# --- 사용자 입력 받기 ---
print("\n은하의 특징을 나타내는 6가지 값을 입력해주세요.")
sersic_input = float(input("1. 세르식 지수 (타원은하 ~4, 나선은하 ~1): "))
ba_ratio_input = float(input("2. 장축 대 단축 비율 (둥글수록 1, 납작할수록 0): "))
sigma_input = float(input("3. 중심 속도 분산 (타원은하 ~200, 나선은하 ~70): "))
sfr_input = float(input("4. 총 별 형성률 (타원은하 <0.01, 나선/불규칙 >0.01): "))
redshift_input = float(input("5. 적색편이 (거리가 멀수록 큼, 예: 0.1): "))
sb_1re_input = float(input("6. 표면 밝기 (SB_1RE) (밝을수록 작음, 예: 0.4): "))


# --- 입력된 값으로 예측 실행 ---
galayT, galaySize = predict_galaxy_all(sersic_input, ba_ratio_input, sigma_input, sfr_input, redshift_input, sb_1re_input)

>>> 0단계: 라이브러리 불러오기 및 필수 라이브러리 설치...
'scikit-learn' 라이브러리가 없어 설치합니다...
'imbalanced-learn' 라이브러리가 없어 설치합니다...

라이브러리 로드 및 확인 완료.

>>> 1단계: dapall.fits 파일을 다운로드하고 데이터를 통합합니다...
 - 'dapall-v3_1_1-3.1.0.fits' 파일이 이미 존재합니다.
데이터 로드 및 통합 성공.

>>> 2단계: 피처 및 라벨 준비...
피처 및 라벨 준비 완료.

>>> 3단계: 2개의 모델을 학습합니다...
  - 모델 1 (모양 예측) 학습 중...
  - 오버샘플링 전 훈련 데이터 클래스 분포:
galaxy_type
Irregular                  5097
Late-type (Spiral)         3157
Early-type (Elliptical)     834
Name: count, dtype: int64

  - 'Early-type (Elliptical)' 클래스를 834개 -> 2502개로 증강합니다.

  - 오버샘플링 후 훈련 데이터 클래스 분포:
galaxy_type
Irregular                  5097
Late-type (Spiral)         3157
Early-type (Elliptical)    2502
Name: count, dtype: int64

  - 모델 1 학습 완료. 테스트 정확도: 0.9996
  - 상세 분류 리포트:
                          precision    recall  f1-score   support

Early-type (Elliptical)       1.00      1.00      1.00       208
              Irregular       1.00      1.00      1.00      1275
     Late-type (Spiral)       1.00      1.00    

In [ ]:
# 필요 라이브러리 설치 (Colab 기준)
!pip install astropy matplotlib

import pandas as pd
import numpy as np
from astropy.io import fits # FITS 파일 처리를 위해 필요
import matplotlib.pyplot as plt

# 1. 은하 정보 (형태, Re, FITS 파일명)
data = [
    ["Late-type (Spiral)","M51",47,"sample_m51.fits"],
    ["Late-type (Spiral)","M63",30,"sample_m63.fits"],
    ["Late-type (Spiral)","NGC 5866",26,"sample_ngc 5866.fits"],
    ["Late-type (Spiral)","NGC 1300",18,"sample_ngc 1300.fits"],
    ["Late-type (Spiral)","NGC 7479",13,"sample_ngc 7479.fits"],
    ["Early-type (Elliptical)","M87",72,"sample_m87.fits"],
    ["Early-type (Elliptical)","M49",50,"sample_m49.fits"],
    ["Early-type (Elliptical)","M104",38,"sample_m104.fits"],
    ["Early-type (Elliptical)","M59",21,"sample_m59.fits"],
    ["Irregular","NGC 6822",105,"sample_ngc 6822.fits"],
    ["Irregular","M82",35,"sample_m82.fits"],
    ["Irregular","NGC 5204",13,"sample_ngc 5204.fits"]
]

df = pd.DataFrame(data, columns=["형태","은하","Re","FITS"])

# 2. 가장 근접한 FITS 파일 검색 함수
def find_closest_fits(input_type, input_re):
    subset = df[df['형태'] == input_type].copy()  # copy()로 경고 제거
    if subset.empty:
        print("해당 형태의 은하가 없습니다.")
        return None
    subset['diff'] = np.abs(subset['Re'] - input_re)
    closest = subset.loc[subset['diff'].idxmin()]
    return closest['FITS']

# 3. 사용자 입력
input_type = galayT
input_re = galaySize

# 4. 가장 근접 FITS 파일 찾기
closest_file = find_closest_fits(input_type, input_re)
if closest_file is None:
    exit()
print("가장 근접한 FITS 파일:", closest_file)

# 5. FITS 파일 로드 및 시각화
try:
    hdu = fits.open(closest_file)
except FileNotFoundError:
    print(f"{closest_file} 파일을 찾을 수 없습니다. Colab에 업로드했는지 확인하세요.")


가장 근접한 FITS 파일: sample_ngc 7479.fits
